In [1]:
import os
import json
import pandas as pd
import numpy as np
from urllib.parse import urlencode
import requests
import codecs
import time
import datetime

In [2]:
from DataRead import Read
from Tactic import MACD, SMA

In [17]:
read=Read()
kline=read.read_kline(freq_type='1d',market='EOSUSDT')
short_flag = True

macd=MACD()
df_ema=macd.ema_tac(df=kline,short=12,long=26,median=9, enable_short=short_flag)
df_dea=macd.dea_tac(df=kline,short=12, long=26, median=9,enable_short=short_flag)

sma=SMA()
df_sma=sma.sma_tac(df=kline, sma1=12, sma2=26,enable_short=short_flag)

window=200
df_ema_test=df_ema.tail(window).copy()
df_dea_test=df_dea.tail(window).copy()
df_sma_test=df_sma.tail(window).copy()

ema={
    'tac_name':'ema',
    'price_return':macd.base_return(df_ema_test)-1,
    'strategy_return':macd.tac_return(df_ema_test)-1,
    'price_drawdown':macd.max_price_drop(df_ema_test),
    'strategy_drawdown':macd.max_tac_drawdown(df_ema_test),
    
}

dea={
    'tac_name':'dea',
    'price_return':macd.base_return(df_dea_test)-1,
    'strategy_return':macd.tac_return(df_dea_test)-1,
    'price_drawdown':macd.max_price_drop(df_dea_test),
    'strategy_drawdown':macd.max_tac_drawdown(df_dea_test),
    
}

sma={
    'tac_name':'sma',
    'price_return':sma.base_return(df_sma_test)-1,
    'strategy_return':sma.tac_return(df_sma_test)-1,
    'price_drawdown':sma.max_price_drop(df_sma_test),
    'strategy_drawdown':sma.max_tac_drawdown(df_sma_test),
    
}

df=pd.DataFrame([ema,dea,sma])

In [18]:
df

,tac_name,price_return,strategy_return,price_drawdown,strategy_drawdown
0,ema,-0.351107,0.794695,-0.5992,-0.4316
1,dea,-0.351107,0.174013,-0.5992,-0.1664
2,sma,-0.351107,0.420025,-0.5992,-0.3800


In [5]:
# 格式输出
# print('{:^10}|{:^20}|{:^20}'.format('Tac','Price Return','Strategy Return'))
# print('{:^10}|{:^20.4f}|{:^20.4f}'.format('EMA',macd.base_return(df_ema_test)-1,macd.tac_return(df_ema_test)-1))
# print('{:^10}|{:^20.4f}|{:^20.4f}'.format('DEA',macd.base_return(df_dea_test)-1,macd.tac_return(df_dea_test)-1))
# print('{:^10}|{:^20.4f}|{:^20.4f}'.format('SMA',macd.base_return(df_sma_test)-1,macd.tac_return(df_sma_test)-1))

In [6]:
# 统计每个策略下交易次数
for test in [df_ema_test, df_dea_test, df_sma_test]:
    test['pos_chg'] = test['Position'] - test['Position'].shift(1)

In [7]:
df_dea_test.groupby(by='pos_chg')['Position'].count()

pos_chg
-2.0      1
-1.0     13
 0.0    271
 1.0     14
Name: Position, dtype: int64

In [8]:
df_dea_test.to_excel('dea.xlsx')